In [1]:
import numpy as np
import pandas as pd

from helper import *

In [2]:
pd.set_option('display.max_columns', None)

In [5]:
path_ass_hist = "./assets/Property_Assessment_History.csv"
path_ass_cur = "./assets/Property_Assessments_Current.csv"
path_geo = "./assets/Parcel_Geographic_Summary.csv"
path_prop_type = "./assets/prop_class.csv"

In [6]:
df_ass_hist = pd.read_csv(path_ass_hist, header=0, low_memory=False)
df_ass_cur = pd.read_csv(path_ass_cur, header=0, low_memory=False)
df_geo = pd.read_csv(path_geo, header=0, low_memory=False)
df_pt = pd.read_csv(path_prop_type, header=0, low_memory=False)

In [7]:
keep_cols_ass_hist = ['PIN','BUILDINGS_ON_PROPERTY_COUNT','STORIES','YEAR_BUILT',
                      'SQFT_OF_FINISHED_LIVING_AREA']
keep_cols_ass_cur = ['PIN','ADDRESS','LAND_SQFT','ASSESSMENT_DATE','LAND_VALUE','DWELLING_VALUE',
                     'TOTAL_VALUE','PROPERTY_CLASSCODE','PROPERTY_CLASS_DESCRIPTION']
keep_cols_geo = ['PIN_PIN','CARE_Name','EZ_Name','NGHOOD_Name','ZONING_Name','Location 1']

In [8]:
df_ass_hist = df_ass_hist[keep_cols_ass_hist].drop_duplicates(keep='first')
df_ass_cur = df_ass_cur[keep_cols_ass_cur].drop_duplicates(keep='first')
df_geo = df_geo[keep_cols_geo].drop_duplicates(keep='first')

In [9]:
df_ass = pd.merge(df_ass_cur,df_ass_hist, left_on=['PIN'], right_on=['PIN'], how="left").drop_duplicates(keep='first')

In [10]:
df = pd.merge(df_ass,df_geo, left_on=['PIN'], right_on=['PIN_PIN'], how="left").drop_duplicates(keep='first')

In [16]:
df =  pd.merge(df,df_pt, left_on=['PROPERTY_CLASS_DESCRIPTION'], right_on=['property_class'], how="left")

In [19]:
df['LAND_VALUE'] = df['LAND_VALUE'].str.slice(1)
df['DWELLING_VALUE'] = df['DWELLING_VALUE'].str.slice(1)
df['TOTAL_VALUE'] = df['TOTAL_VALUE'].str.slice(1)
df['year'] = df['ASSESSMENT_DATE'].str.slice(6,10)
df['ez_flag'] = ~df['EZ_Name'].isnull()
df['care_flag'] = ~df['CARE_Name'].isnull()
df['lat']=df['Location 1'].str.slice(1,10)
df['lon']=df['Location 1'].str.slice(11,-1)
df = changeToNumeric(df, ['year', 'LAND_VALUE', 'DWELLING_VALUE', 'TOTAL_VALUE', 'STORIES','lat','lon'])

In [20]:
df = df.drop(['PIN_PIN','Location 1','ASSESSMENT_DATE','property_class'],axis=1)

In [21]:
df['SQFT_OF_FINISHED_LIVING_AREA'] = df['SQFT_OF_FINISHED_LIVING_AREA'].fillna(0)

In [22]:
df = df.sort_values(['SQFT_OF_FINISHED_LIVING_AREA'], ascending=False)\
    .drop_duplicates('PIN',keep="first").reset_index(drop=True)

In [23]:
df.YEAR_BUILT[df.YEAR_BUILT == 1625] = 1925

/home/connor/apps/anaconda3/envs/realestate/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df.lon[df.lon > 0] = -df.lon
df.lon[df.lon > -1] = df.lon - 77
df.lon[df.lon > -8] = df.lon - 70

/home/connor/apps/anaconda3/envs/realestate/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/connor/apps/anaconda3/envs/realestate/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/connor/apps/anaconda3/envs/realestate/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is sepa

In [25]:
savePickle(df, pickleName="rva_bldgs_nov_2017.p")

In [26]:
df_old = df[df['YEAR_BUILT']<=1920] 
df_old = df_old[df_old['YEAR_BUILT'] > 1191]

In [27]:
savePickle(df_old, pickleName="df_old.p")

In [29]:
df_old['YEAR_BUILT'].nsmallest(5)

918      1720.0
11937    1725.0
2677     1735.0
46516    1737.0
4387     1745.0
Name: YEAR_BUILT, dtype: float64

In [ ]:
df_old[df_old["YEAR_BUILT"]==1735.0]